# Interactive Visualization of CheMin Mineral Data on Mars

This Python Dash app visualizes mineral abundances measured by the CheMin instrument aboard the Curiosity rover.
It provides interactive plots and maps using preprocessed datasets:
- `combined_chemin_data_all_sols.csv`
- `combined_sol_data.csv`


## Load Combined CSV Files

In [3]:
import dash
import dash_leaflet as dl
from dash import html, dcc
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Load preprocessed data (must be placed in ./data/)
sol_coords_elevation = pd.read_csv('./data/combined_sol_data.csv')
sols_all_data = pd.read_csv('./data/combined_chemin_data_all_sols.csv')

# Create Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Sols on Mars with Mineral Data and Geographical Information"),
    
    dcc.Dropdown(
        id='sol-dropdown',
        options=[{'label': f'Sol {sol}', 'value': sol} for sol in sols_all_data['sol'].unique()],
        value=sols_all_data['sol'].unique()[0],
        clearable=False,
        style={'width': '50%'}
    ),
    
    dcc.Graph(id='mineral-abundance-plot'),
    
    html.Div(id='geo-info', style={'padding': '20px', 'fontSize': '16px'}),
    
    dl.Map([
        dl.ImageOverlay(
            url="https://corsproxy.io/?https://astrogeology.usgs.gov/ckan/dataset/06137610-fb79-4b45-a5ef-63c4869d48b2/resource/dfb28a0a-dd06-40c9-b7e2-2c7d79e0487a/download/msl_gale_orthophoto_mosaic_1024.jpg",
            bounds=[[-5.249525, 137.094719], [-4.068583, 137.853896]],
            opacity=0.8
        ),
        dl.LayerGroup(id="layer")
    ], style={'width': '100%', 'height': '500px'}, center=[-4.5895, 137.4417], zoom=10)
])


## Callback Logic

In [6]:
@app.callback(
    [Output('mineral-abundance-plot', 'figure'),
     Output('geo-info', 'children'),
     Output('layer', 'children')],
    [Input('sol-dropdown', 'value')]
)
def update_graph_and_map(selected_sol):
    filtered_data = sols_all_data[sols_all_data['sol'] == selected_sol]
    
    fig = px.scatter(
        filtered_data,
        x='MINERAL',
        y='PERCENT',
        error_y='ERROR',
        title=f"Mineral Abundance for Sol {selected_sol}",
        labels={'PERCENT': 'Abundance (%)', 'MINERAL': 'Minerals'}
    )
    fig.update_traces(marker=dict(color='blue', size=10, line=dict(width=1, color='DarkSlateGrey')))
    fig.update_layout(showlegend=False, xaxis_title="Mineral", yaxis_title="Mineral Abundance (%)",
                      font=dict(size=14), margin=dict(l=40, r=40, t=50, b=100), xaxis_tickangle=45)

    geo_info = sol_coords_elevation[sol_coords_elevation['sol'] == selected_sol]
    if not geo_info.empty:
        lat, lon, elev = geo_info.iloc[0][['latitude', 'longitude', 'elevation']]
        geo_text = f"Latitude: {lat}, Longitude: {lon}, Elevation: {elev} m"
        markers = [dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(f"Sol {selected_sol}"),
            dl.Popup(f"Sol {selected_sol}<br>Elevation: {elev} m")
        ])]
    else:
        geo_text = "No geographical information available for this sol."
        markers = []
    
    return fig, geo_text, markers


## Run the App

In [11]:
if __name__ == "__main__":
    app.run_server(debug=True, port=8052)
